In [1]:
import os
import numpy as np
from tqdm import tqdm

from lib.utils import model_results_from_npz
from lib.projection import gram_schmidt, project_confidences
from lib.redistribute import redistribute_confidences_of_class

# ===== Dataset Config =====
# If you want explicit class names, load from your CIFAR-100 mapping.
NUM_CLASSES = 100

SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]   # example seeds
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

# ===== Paths =====
DIR_ORI = "../results/cifar100/"
DIR_OUT = "../results/redis_cifar100/"
os.makedirs(DIR_OUT, exist_ok=True)

# ===== Redistribute Loop =====
for seed in SEEDS:
    file = f"cifar100_resnet18_s{seed}_e{EPOCHS}.npz"
    filepath = os.path.join(DIR_ORI, file)
    if not os.path.exists(filepath):
        print(f"Skip missing: {filepath}")
        continue

    # Load original once per seed
    model_ori = model_results_from_npz(filepath=filepath, num_classes=NUM_CLASSES)

    # Precompute averages & projections once
    avg_confidences = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.float32)
    for i in range(NUM_CLASSES):
        avg_confidences[i] = np.mean(model_ori.confidences[i], axis=0)

    avg_confidences_ortho = gram_schmidt(avg_confidences)

    for removed_class in tqdm(REMOVED_CLASSES, desc=f"Seed {seed}", leave=False):
        proj_confidences = project_confidences(
            model_ori.confidences,
            avg_confidences_ortho,
            removed_class
        )

        avg_proj_confidences = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.float32)
        for i in range(NUM_CLASSES):
            avg_proj_confidences[i] = np.mean(proj_confidences[i], axis=0)

        # Redistribute per class
        redis = {}
        for cls in range(NUM_CLASSES):
            redis[cls] = redistribute_confidences_of_class(
                conf=model_ori.confidences[cls],
                proj_conf=proj_confidences[cls],
                avg_proj_conf=avg_proj_confidences[cls],
                removed_class=removed_class,
            )

        # Concatenate back
        all_confs = np.vstack([redis[cls] for cls in range(NUM_CLASSES)])
        all_targets = np.concatenate([
            np.full(redis[cls].shape[0], cls, dtype=int) for cls in range(NUM_CLASSES)
        ])
        all_preds = np.argmax(all_confs, axis=1)


        # Save
        out_path = os.path.join(DIR_OUT, f"cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz")
        np.savez_compressed(out_path, preds=all_preds, targets=all_targets, confs=all_confs)
        print(f"[Saved] {out_path}")


Seed 42:   5%|▌         | 2/40 [00:00<00:07,  5.00it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd4.npz
[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd7.npz


Seed 42:  10%|█         | 4/40 [00:00<00:06,  5.46it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd6.npz
[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd11.npz


Seed 42:  15%|█▌        | 6/40 [00:01<00:05,  6.17it/s]

...

Seed 93:  90%|█████████ | 36/40 [00:06<00:00,  5.98it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd88.npz
[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd85.npz


Seed 93:  95%|█████████▌| 38/40 [00:06<00:00,  5.98it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd92.npz
[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd94.npz


[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd99.npz
[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd97.npz
